# Visualize decay topologies

In [ ]:
# To run in in Google Colab, uncomment the following:

# !pip install expertsystem graphviz

The {mod}`~expertsystem.io` module allows you to convert {class}`.StateTransitionGraph` and {class}`.Topology` instances to [DOT language](https://graphviz.org/doc/info/lang.html) with {func}`.asdot`. You can visualize its output with third-party libraries, such as [Graphviz](https://graphviz.org). This is particularly useful after running {meth}`~.StateTransitionManager.find_solutions`, which produces a {class}`.Result` object with a {class}`.list` of {class}`.StateTransitionGraph` instances (see {doc}`reaction`).

## Topologies

First of all, here are is an example of how to visualize a group of {class}`.Topology` instances. We use {func}`.create_isobar_topologies` and {func}`.create_n_body_topology` to create a few standard topologies.

In [ ]:
import graphviz
from expertsystem import io
from expertsystem.reaction.topology import (
    create_isobar_topologies,
    create_n_body_topology,
)

In [ ]:
topology = create_n_body_topology(2, 4)
graphviz.Source(io.asdot(topology))

Note the IDs of the {attr}`~.Topology.nodes` is also rendered if there is more than node:

In [ ]:
topologies = create_isobar_topologies(4)
graphviz.Source(io.asdot(topologies))

This can be turned off with the arguments of {func}`.asdot`:

In [ ]:
topologies = create_isobar_topologies(3)
graphviz.Source(io.asdot(topologies, render_node=False))

## {class}`.StateTransitionGraph`s

Here, we'll visualize the allowed transitions for the decay $\psi' \to \gamma\eta\eta$ as an example.

In [ ]:
import expertsystem as es

result = es.reaction.generate(
    initial_state="psi(2S)",
    final_state=["gamma", "eta", "eta"],
    allowed_interaction_types="EM",
)

As noted in {ref}`usage/reaction:3. Find solutions`, the {attr}`~.Result.transitions` contain all spin projection combinations (which is necessary for the {mod}`~expertsystem.amplitude` module). It is possible to convert all these solutions to DOT language with {func}`~.asdot`. To avoid visualizing all solutions, we just take a subset of the {attr}`~.Result.transitions`:

In [ ]:
dot = es.io.asdot(result.transitions[::50][:3])  # just some selection

This {class}`str` of [DOT language](https://graphviz.org/doc/info/lang.html) for the list of {class}`.StateTransitionGraph` instances can then be visualized with a third-party library, for instance, with {class}`graphviz.Source`:

````{margin}
```{warning}
[graphviz](graphviz.Source) requires your system to have DOT installed, see {doc}`Installation <graphviz:index>`.
```
````

In [ ]:
import graphviz

graphviz.Source(dot)

You can also serialize the DOT string to file with [io.write](expertsystem.io.write). The file extension for a DOT file is `.gv`:

In [ ]:
es.io.write(result, "decay_topologies_with_spin.gv")

### Collapse graphs

Since this list of all possible spin projections {attr}`~.Result.transitions` is rather long, it is often useful to use `strip_spin=True` or `collapse_graphs=True` to bundle comparable graphs. First, {code}`strip_spin=True` allows one collapse (ignore) the spin projections (we again show a selection only):

In [ ]:
dot = es.io.asdot(
    result.transitions[:3],
    render_edge_id=False,
    strip_spin=True,
)
graphviz.Source(dot)

```{note}
By default, `.asdot` renders edge IDs, because they represent the (final) state IDs as well. In the example above, we switched this off.
```

If that list is still too much, there is {code}`collapse_graphs=True`, which bundles all graphs with the same final state groupings:

In [ ]:
dot = es.io.asdot(result, collapse_graphs=True, render_node=False)
graphviz.Source(dot)